# Ex 1.1 Motion Detection & Tracking
This task focuses on developing an application to detect and track moving cars from a camera recording. The algorithm of the application is based on the frame differencing and background subtraction techniques. The application will only focus on detecting cars that are in the "Main street".

In [1]:
# IMPORTING LIBRARIES FOR PROJECT
import cv2
import numpy as np

The traffic video ```Traffic_Laramie_1.mp4``` is loaded for verifying results.

In [2]:
# loading the video file
vid_path = "media/Traffic_Laramie_1.mp4"
capture = cv2.VideoCapture(vid_path)

# checking if video is opened successfully
if not capture.isOpened(): print("Error: Could not open video.")
else: print("Successfully opened video.")

Successfully opened video.


OpenCV will be used to implement the background subtraction in the algorithm.

This project will use the updated ```createBackgroundSubtractorMOG2()```
- ```history``` is the length of the history
- ```varThreshold``` is the threshold on the squared Mahalanobis distance between the pixel and the model to decide whether a pixel is well described by the background model
- ```detectShadows``` checks for shadows and marks them

The base values are used, and will be modified to better detect the cars.

In [3]:
# creating an instance of Gaussian Mixture-based background subtraction
background_sub = cv2.createBackgroundSubtractorMOG2( history = 500,
                                                     varThreshold = 50,
                                                     detectShadows = True )

Now, the algorithm will read the input video frame-by-frame, and apply background subtraction to extract the moving objects. Morphological operations will be done to clean the mask, and the algorithm will detect contours to find moving cars. This is then displayed by drawing a bounding box around detected cars.

To ensure that only vehicles are detected, the proportion of height to width will be checked. Since vehicles have a larger width than height, this would easily rule out detected human movement. This works since we are only focusing on the main street, so all cars will have a larger width ratio in comparison with height.

After displaying the results, it is recognised that the movement detection tends to be very fragmented. As such, adding ```cv2.dilate()``` ensures that it expands white areas in the detected objects to help connect broken parts of a vehicle. ```cv2.erode()``` is then used to remove small noise and false detections. This refines the object's boundaries.

In [4]:
# defining main street region of interest (ROI)
roi_top_left = (0, 250)
roi_bot_right = (1040, 600)

while True:
    check, img = capture.read()
    if not check: break # stop if end of video
        
    # applying background subtraction
    fg_mask = background_sub.apply(img)
    # dilating white areas of detected object
    fg_mask = cv2.dilate(fg_mask, None, 20)
    # shrinking white areas in binary mask
    fg_mask = cv2.erode(fg_mask, None, 15)

    # applying thresholding to clean up noise
    threshold_mask = cv2.threshold(fg_mask, 150, 255, cv2.THRESH_BINARY)[1]
    
    # applying ROI (main street)
    roi = np.zeros_like(threshold_mask)
    cv2.rectangle(roi, roi_top_left, roi_bot_right, 255, thickness = -1)
    threshold_mask = cv2.bitwise_and(threshold_mask, roi)
    
    # finding contours to detect moving objects
    contours = cv2.findContours(threshold_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    for c in contours:
        # drawing bounding box around detected cars
        x, y, w, h = cv2.boundingRect(c)
        # filtering out human shapes or small contours
        if (h/w > 1.3) | (cv2.contourArea(c) < 1100): continue
        # drawing separate bounding boxes for each car
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # draw ROI rectangle on original frame
    cv2.rectangle(img, roi_top_left, roi_bot_right, (0, 0, 255), 2)
    
    # displaying the results
    cv2.imshow("Traffic Video", img)
    
    # exiting the loop
    key = cv2.waitKey(1)
    # using "q" key to exit
    if key == ord("q"):
        break

# after the loop, release the video object
capture.release()
# destroy all windows
cv2.destroyAllWindows()